In [1]:
from ltlcross_runner import LtlcrossRunner, bogus_to_lcr

# Print ltlcross command
* For bug hunting
* print command into the log-file

In [2]:
%run Tools.ipynb

In [3]:
runners = {}
cols=["states","transitions","acc","time","nondet_states"]
for source in ('literature','random'):
    for t in ('full','ltl3dra'):
        name = '{}_{}'.format(source,t)
        tools = full_tools if t=='full' else ltl3dra_tools
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data_12_28/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

name = 'random_fg'
tools = ltl3dra_tools
runners[name] = LtlcrossRunner(tools,\
                    res_filename='data_12_28/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

## Bogus_to_lcr
* ltlcross stores formulae in the form `Ga | Fb` in the `_bogus` files. However, in the `.csv` files it uses `p0` AP.

In [4]:
import subprocess

In [5]:
r = !cat data/random_full_bogus.ltl | head -n 1
f = r[0]; f
!ltlfilt -r0 -p --relabel=pnn -f '{f}'

(((p0) R (p1)) & (G(F(((p1) & (F(p2))) | ((!(p1)) & (G(!(p2)))))))) | ((F(G(((p1) & (G(!(p2)))) | ((!(p1)) & (F(p2)))))) & ((!(p0)) U (!(p1))))


In [6]:
args = ['-r0','--relabel=pnn','-f',f]
subprocess.check_output(["ltlfilt"] + args, universal_newlines=True).strip()

'((p0 R p1) & GF((p1 & Fp2) | (!p1 & G!p2))) | (FG((p1 & G!p2) | (!p1 & Fp2)) & (!p0 U !p1))'

In [7]:
bogus_to_lcr(f)

'((p0 R p1) & GF((p1 & Fp2) | (!p1 & G!p2))) | (FG((p1 & G!p2) | (!p1 & Fp2)) & (!p0 U !p1))'

## id_of_form
In bogus files we get formulas. Let's get their id.

In [8]:
r = runners['random_full']
r.parse_results()

In [9]:
f3 = 'a R XX(Ga R b)'

In [10]:
r.id_of_form(f3, True)

236

In [11]:
r.form_of_id(r.id_of_form(f3,True))

p0 R XX(Gp0 R p1)

In [12]:
for i in range(10):
    print(r.id_of_form(r.form_of_id(i,False)))

0
1
2
3
4
5
6
7
8
9


## Mark false
* I want to mark false results that I know are not correct
* Marking false means that states etc. are set to 0 and we use special column ``incorrect`` and set it to True.

In [13]:
import pandas as pd
from experiments_lib import pretty_print

In [14]:
r.mark_incorrect(0,'R3')
r.mark_incorrect(0,'R4')
#r.mark_incorrect(1,'R3')
#r.mark_incorrect(1,'R4')
#r.mark_incorrect(2,'R3')
#r.mark_incorrect(0,'ltl2tgba')

In [15]:
r.incorrect.sum().head(n=3)

tool
R3         3
R3_dsra    0
R4         2
dtype: int64

In [16]:
r.incorrect.head(n=3)

,tool,R3,R3_dsra,R4,R4_dsra,Spot+LTL3BA,Spot+LTL3BAd,Spot+Spot,SpotG+LTL3BA_gen,SpotG+LTL3BAd_gen,SpotG+LTL3TELA,...,devltl2tgba,ltl2dpa(Rab),ltl2dpa(ldba),ltl2dstar+LTL3BA,ltl2dstar+LTL3BAd,ltl2dstar+Spot,ltl2dstarNBA+LTL3BA,ltl2dstarNBA+LTL3BAd,ltl2dstarNBA+Spot,ltl2tgba
form_id,formula,,,,,,,,,,,,,,,,,,,,,
0,(p0 R p1) R Fp2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,X((Fp0 | Gp1) R X(!p0 | p2)),True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,(p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) | (((!p0 & (!p2 | !p3)) R X(p1 & !p3)) & (!p0 | G!p1)),True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
r.values.head(n=3)

column                                                          time  \
tool                                                              R3   
form_id formula                                                        
0       (p0 R p1) R Fp2                                     0.112112   
1       X((Fp0 | Gp1) R X(!p0 | p2))                        0.133010   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...  0.144385   

column                                                                \
tool                                                         R3_dsra   
form_id formula                                                        
0       (p0 R p1) R Fp2                                     0.147781   
1       X((Fp0 | Gp1) R X(!p0 | p2))                        0.151969   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...  0.175262   

column                                                                \
tool                                                              R4   
form_id formula                                                        
0       (p0 R p1) R Fp2                                     0.537234   
1       X((Fp0 | Gp1) R X(!p0 | p2))                        0.681558   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...  0.684866   

column                                                                \
tool                                                         R4_dsra   
form_id formula                                                        
0       (p0 R p1) R Fp2                                     0.494911   
1       X((Fp0 | Gp1) R X(!p0 | p2))                        0.782628   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...  0.767024   

column                                                                  \
tool                                                       Spot+LTL3BA   
form_id formula                                                          
0       (p0 R p1) R Fp2                                       0.011653   
1       X((Fp0 | Gp1) R X(!p0 | p2))                          0.010077   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...    0.015304   

column                                                                   \
tool                                                       Spot+LTL3BAd   
form_id formula                                                           
0       (p0 R p1) R Fp2                                        0.008478   
1       X((Fp0 | Gp1) R X(!p0 | p2))                           0.009060   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...     0.014918   

column                                                                \
tool                                                       Spot+Spot   
form_id formula                                                        
0       (p0 R p1) R Fp2                                     0.009685   
1       X((Fp0 | Gp1) R X(!p0 | p2))                        0.008638   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...  0.010820   

column                                                                       \
tool                                                       SpotG+LTL3BA_gen   
form_id formula                                                               
0       (p0 R p1) R Fp2                                            0.009679   
1       X((Fp0 | Gp1) R X(!p0 | p2))                               0.012384   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...         0.016046   

column                                                                        \
tool                                                       SpotG+LTL3BAd_gen   
form_id formula                                                                
0       (p0 R p1) R Fp2                                             0.007278   
1       X((Fp0 | Gp1) R X(!p0 | p2))                                0.013499   
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...      

In [18]:
r.cummulative().head()

tool
R3              4239.0
R3_dsra        10213.0
R4              3681.0
R4_dsra         5618.0
Spot+LTL3BA     4906.0
dtype: float64

In [19]:
r.na_incorrect()
r.values.states.head(n=3)

,tool,R3,R3_dsra,R4,R4_dsra,Spot+LTL3BA,Spot+LTL3BAd,Spot+Spot,SpotG+LTL3BA_gen,SpotG+LTL3BAd_gen,SpotG+LTL3TELA,...,devltl2tgba,ltl2dpa(Rab),ltl2dpa(ldba),ltl2dstar+LTL3BA,ltl2dstar+LTL3BAd,ltl2dstar+Spot,ltl2dstarNBA+LTL3BA,ltl2dstarNBA+LTL3BAd,ltl2dstarNBA+Spot,ltl2tgba
form_id,formula,,,,,,,,,,,,,,,,,,,,,
0,(p0 R p1) R Fp2,NaN,10.0,NaN,6.0,6.0,6.0,6.0,4.0,4.0,4.0,...,4.0,4.0,4.0,6.0,6.0,8.0,12.0,12.0,12.0,NaN
1,X((Fp0 | Gp1) R X(!p0 | p2)),NaN,18.0,NaN,9.0,6.0,6.0,4.0,6.0,6.0,4.0,...,4.0,9.0,7.0,8.0,8.0,5.0,8.0,8.0,5.0,4.0
2,(p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) | (((!p0 & (!p2 | !p3)) R X(p1 & !p3)) & (!p0 | G!p1)),NaN,17.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,8.0,8.0,11.0,11.0,11.0,11.0,11.0,11.0,8.0


In [20]:
r.cummulative().head()

tool
R3              4217.0
R3_dsra        10168.0
R4              3660.0
R4_dsra         5595.0
Spot+LTL3BA     4886.0
dtype: float64

In [21]:
r.incorrect.sum()

tool
R3                      3
R3_dsra                 0
R4                      2
R4_dsra                 0
Spot+LTL3BA             0
Spot+LTL3BAd            0
Spot+Spot               0
SpotG+LTL3BA_gen        0
SpotG+LTL3BAd_gen       0
SpotG+LTL3TELA          0
SpotG+Spot_gen          0
devSpot+LTL3BA          0
devSpot+LTL3BAd         0
devSpot+Spot            0
devSpotG+LTL3BA_gen     0
devSpotG+LTL3BAd_gen    0
devSpotG+LTL3TELA       0
devSpotG+Spot_gen       0
devltl2tgba             0
ltl2dpa(Rab)            0
ltl2dpa(ldba)           0
ltl2dstar+LTL3BA        0
ltl2dstar+LTL3BAd       0
ltl2dstar+Spot          0
ltl2dstarNBA+LTL3BA     0
ltl2dstarNBA+LTL3BAd    0
ltl2dstarNBA+Spot       0
ltl2tgba                1
dtype: int64

In [22]:
r.values.states[r.values.states.isnull().any(axis=1)]

,tool,R3,R3_dsra,R4,R4_dsra,Spot+LTL3BA,Spot+LTL3BAd,Spot+Spot,SpotG+LTL3BA_gen,SpotG+LTL3BAd_gen,SpotG+LTL3TELA,...,devltl2tgba,ltl2dpa(Rab),ltl2dpa(ldba),ltl2dstar+LTL3BA,ltl2dstar+LTL3BAd,ltl2dstar+Spot,ltl2dstarNBA+LTL3BA,ltl2dstarNBA+LTL3BAd,ltl2dstarNBA+Spot,ltl2tgba
form_id,formula,,,,,,,,,,,,,,,,,,,,,
0,(p0 R p1) R Fp2,NaN,10.0,NaN,6.0,6.0,6.0,6.0,4.0,4.0,4.0,...,4.0,4.0,4.0,6.0,6.0,8.0,12.0,12.0,12.0,NaN
1,X((Fp0 | Gp1) R X(!p0 | p2)),NaN,18.0,NaN,9.0,6.0,6.0,4.0,6.0,6.0,4.0,...,4.0,9.0,7.0,8.0,8.0,5.0,8.0,8.0,5.0,4.0
2,(p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) | (((!p0 & (!p2 | !p3)) R X(p1 & !p3)) & (!p0 | G!p1)),NaN,17.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,8.0,8.0,11.0,11.0,11.0,11.0,11.0,11.0,8.0
33,((p0 R p1) & GF((p1 & Fp2) | (!p1 & G!p2))) | (FG((p1 & G!p2) | (!p1 & Fp2)) & (!p0 U !p1)),NaN,NaN,3.0,18.0,34.0,23.0,12.0,9.0,9.0,27.0,...,8.0,6.0,5.0,18.0,18.0,16.0,62.0,38.0,29.0,8.0
41,X((Gp0 & F(F(p1 R p2) R p3)) | (F!p0 & G(G(!p1 U !p2) U !p3))),NaN,NaN,8.0,15.0,12.0,10.0,10.0,9.0,9.0,10.0,...,9.0,14.0,16.0,272.0,195.0,195.0,271.0,195.0,195.0,9.0
44,XG((Fp0 & (((p2 | Xp3) & XF!p1) | (!p2 & X(!p3 & Gp1)))) | (G!p0 & (((p2 | Xp3) & XGp1) | (!p2 & X(!p3 & F!p1))))),253.0,NaN,62.0,81.0,36.0,29.0,27.0,26.0,26.0,24.0,...,25.0,62.0,33.0,55.0,44.0,41.0,55.0,44.0,41.0,25.0
57,((!p0 | p1) & G((!p2 U Gp1) U (!p3 & p4))) | (p0 & !p1 & F((p2 R F!p1) R (p3 | !p4))),17.0,NaN,10.0,17.0,17.0,17.0,14.0,11.0,11.0,16.0,...,11.0,10.0,12.0,26.0,32.0,32.0,70.0,82.0,86.0,11.0
73,Gp0 | X(Xp1 R ((Fp2 & Gp3) | (G!p2 & F!p3))),NaN,NaN,27.0,25.0,26.0,24.0,25.0,24.0,24.0,24.0,...,24.0,24.0,27.0,121.0,49.0,58.0,185.0,49.0,58.0,24.0
106,((p0 & F!p1) | (!p0 & Gp1)) R (XFp1 & ((p2 & Gp0) | (!p2 & F!p0))),38.0,NaN,29.0,32.0,33.0,28.0,29.0,33.0,29.0,32.0,...,28.0,29.0,31.0,113.0,75.0,71.0,113.0,75.0,71.0,31.0


In [23]:
r.exit_status.head(n=3)

,tool,R3,R3_dsra,R4,R4_dsra,Spot+LTL3BA,Spot+LTL3BAd,Spot+Spot,SpotG+LTL3BA_gen,SpotG+LTL3BAd_gen,SpotG+LTL3TELA,...,devltl2tgba,ltl2dpa(Rab),ltl2dpa(ldba),ltl2dstar+LTL3BA,ltl2dstar+LTL3BAd,ltl2dstar+Spot,ltl2dstarNBA+LTL3BA,ltl2dstarNBA+LTL3BAd,ltl2dstarNBA+Spot,ltl2tgba
form_id,formula,,,,,,,,,,,,,,,,,,,,,
0,(p0 R p1) R Fp2,incorrect,ok,incorrect,ok,ok,ok,ok,ok,ok,ok,...,ok,ok,ok,ok,ok,ok,ok,ok,ok,incorrect
1,X((Fp0 | Gp1) R X(!p0 | p2)),incorrect,ok,incorrect,ok,ok,ok,ok,ok,ok,ok,...,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok
2,(p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) | (((!p0 & (!p2 | !p3)) R X(p1 & !p3)) & (!p0 | G!p1)),incorrect,ok,ok,ok,ok,ok,ok,ok,ok,ok,...,ok,ok,ok,ok,ok,ok,ok,ok,ok,ok


## Errors

In [24]:
r.get_error_count('no output')

tool
R3                      0
R3_dsra                 0
R4                      0
R4_dsra                 0
Spot+LTL3BA             0
Spot+LTL3BAd            0
Spot+Spot               0
SpotG+LTL3BA_gen        0
SpotG+LTL3BAd_gen       0
SpotG+LTL3TELA          0
SpotG+Spot_gen          0
devSpot+LTL3BA          0
devSpot+LTL3BAd         0
devSpot+Spot            0
devSpotG+LTL3BA_gen     0
devSpotG+LTL3BAd_gen    0
devSpotG+LTL3TELA       0
devSpotG+Spot_gen       0
devltl2tgba             0
ltl2dpa(Rab)            0
ltl2dpa(ldba)           0
ltl2dstar+LTL3BA        0
ltl2dstar+LTL3BAd       0
ltl2dstar+Spot          0
ltl2dstarNBA+LTL3BA     0
ltl2dstarNBA+LTL3BAd    0
ltl2dstarNBA+Spot       0
ltl2tgba                0
dtype: int64

## Cross_comparison
I want to have a $n\times n$ table that shows how many times the tool in a row gets better result than the one in the column. I would like to make it to take sequence of _properties_ to compare on as an input. In case of the equality of the first one, use the second property etc.

In [25]:
len(r.smaller_than('R4','SpotG+Spot_gen'))

157

In [26]:
r.cross_compare()

,ltl2dstarNBA+LTL3BAd,ltl2dstarNBA+Spot,Spot+LTL3BA,devSpotG+LTL3BAd_gen,ltl2dpa(ldba),Spot+LTL3BAd,devSpotG+Spot_gen,SpotG+Spot_gen,devSpotG+LTL3TELA,devSpotG+LTL3BA_gen,...,SpotG+LTL3TELA,ltl2dstarNBA+LTL3BA,ltl2dpa(Rab),devSpot+Spot,R4,ltl2dstar+LTL3BAd,R4_dsra,ltl2dstar+Spot,R3,devSpot+LTL3BA
ltl2dstarNBA+LTL3BAd,0,70,12,2,24,3,4,4,8,3,...,8,165,37,4,32,22,60,54,137,12
ltl2dstarNBA+Spot,92,0,27,9,30,13,2,2,14,10,...,14,194,47,2,41,78,72,20,150,27
Spot+LTL3BA,471,454,0,30,205,20,31,32,61,33,...,62,484,198,35,131,437,274,426,336,2
devSpotG+LTL3BAd_gen,491,481,212,0,278,185,50,53,127,26,...,128,492,265,179,158,484,363,470,381,212
ltl2dpa(ldba),448,440,218,136,0,203,135,136,137,140,...,138,458,155,189,125,443,309,436,390,218
Spot+LTL3BAd,482,471,106,35,225,0,28,30,72,47,...,72,484,212,34,137,463,309,453,348,105
devSpotG+Spot_gen,486,488,251,102,301,223,0,9,166,115,...,167,489,303,163,165,483,393,484,378,251
SpotG+Spot_gen,486,488,251,102,300,222,1,0,165,114,...,166,489,302,162,164,483,393,484,378,251
devSpotG+LTL3TELA,479,471,197,56,276,164,47,48,0,65,...,8,480,260,153,149,472,352,461,380,196
devSpotG+LTL3BA_gen,488,477,224,48,271,207,81,83,149,0,...,150,492,257,201,175,481,359,465,381,224


#### Better_than
solves multiple comparison criteria

In [27]:
t1 = 'R4'
t2 = 'R3'
props = ('states','acc')#,'transitions','time')

In [28]:
r.better_than(t1,t2)#,['states','acc'])

column                                                     states         acc  \
tool                                                           R3    R4    R3   
form_id formula                                                                 
2       (p0 & Fp1 & ((p0 | (p2 & p3)) U X(!p1 | p3))) |...    NaN   8.0   NaN   
33      ((p0 R p1) & GF((p1 & Fp2) | (!p1 & G!p2))) | (...    NaN   3.0   NaN   
41      X((Gp0 & F(F(p1 R p2) R p3)) | (F!p0 & G(G(!p1 ...    NaN   8.0   NaN   
73      Gp0 | X(Xp1 R ((Fp2 & Gp3) | (G!p2 & F!p3)))          NaN  27.0   NaN   
171     ((p0 & (p0 R Fp1) & X(Fp2 R (p0 | Fp2))) | ((!p...    NaN  72.0   NaN   
202     (Fp0 & FG(((!p1 | !p2) R !p3) U (!p1 R !p4))) |...    NaN   8.0   NaN   
424     (Fp0 & Fp1 & (p1 U Gp2)) | ((G!p1 | G!p0) & (!p...    NaN  10.0   NaN   
3       X(Gp0 R p1)                                           5.0   4.0   4.0   
4       (Gp0 & (p1 R p2)) | (F!p0 & (!p1 U !p2))              6.0   5.0   5.0   
5       (Xp0 & (p0 U p1)) R p2                                7.0   6.0   3.0   
7       (Fp0 & Gp1 & Xp2) R (Gp0 | Xp1)                      12.0  11.0   5.0   
8       XG(p0 & X((p1 U p2) R (p0 | (p1 U p2))))              7.0   3.0  12.0   
9       p0 U (p1 U (p2 R p3))                                 5.0   4.0   3.0   
10      (p0 & (Gp1 U (!p2 | p3))) | (!p0 & (F!p1 R (p2 ...    7.0   6.0   8.0   
14      (Gp0 U p1) | (p2 & X(p0 R (p0 | p2)))                 6.0   5.0   5.0   
15      (Xp0 & ((F(p0 | p1) & (p1 U G!p2)) | ((!p1 R Fp...   14.0  13.0  13.0   
16      X(p0 | G(Xp1 U (((p2 R p3) & G!p2) | (Fp2 & (!p...   35.0  22.0   5.0   
17      ((!p0 & !p1) | (p0 & p1)) & X(G(Xp2 & (p3 R p0)...    9.0   4.0   7.0   
19      (p0 R p1) R p2                                        5.0   4.0   7.0   
20      XG(Fp0 U ((p1 & (p2 | Xp0)) | (!p1 & !p2 & X!p0)))   17.0   7.0   4.0   
22      XX((FGp0 U p1) R p2)                                  8.0   7.0   7.0   
25      (p0 | p1) R (G!p0 U p2)                               7.0   6.0   7.0   
30      X((!p0 & Xp0 & (!p0 R p1)) | ((p0 | X!p0) & (p0...    7.0   6.0   1.0   
31      (Gp0 | (!p1 & (p2 R Fp1)) | (p1 & (!p2 U G!p1))...   47.0  46.0  20.0   
32      (p0 & ((X!p0 U Gp1) R (!p2 | p3))) | (!p0 & ((X...   13.0  12.0  18.0   
34      FG(Fp0 U (Fp1 R p2))                                 18.0   8.0   8.0   
36      G(G(p0 | G!p0) U p1)                                  6.0   5.0   6.0   
37      FG(p0 & (G!p0 U p1))                                  2.0   1.0   2.0   
38      X(p0 | p1 | (p2 & F(p0 | (!p0 R !p2))))               5.0   4.0   3.0   
39      p0 & Xp1 & (Gp2 R p1)                                 5.0   4.0   4.0   
...                                                           ...   ...   ...   
392     ((p0 | p1) & X(p1 R X!p0)) | (!p0 & !p1 & X(!p1...    9.0   9.0   3.0   
394     !p0 | F(((!p1 R !p2) & ((p0 & Fp1) | (!p0 & G!p...    9.0   9.0   7.0   
397     GF(p0 & (Gp0 R p1))                                   5.0   5.0  14.0   
400     XG(p0 | X(p1 R Xp2))                                 13.0  13.0  10.0   
405     X(GFp0 | (Fp1 & (p2 R p3)) | (G!p1 & (!p2 U !p3)))    7.0   7.0  10.0   
408     F(p0 & Fp1) R (p0 & Xp2)                              7.0   7.0   3.0   
409     p0 | F(F!p0 R (p1 | GFp1))                            4.0   4.0   3.0   
411     (p0 & Gp1 & (G!p2 R X!p3)) | (F!p1 & (!p0 | (Fp...   11.0  11.0   7.0   
414     XF(Xp0 & (F(p0 & p1) R p2))                           8.0   8.0   3.0   
427     Fp0 | XG(Fp0 | Xp1)                                   6.0   6.0   5.0   
428     F((FGp0 & (!p1 U ((p0 & !p2) | (!p0 & p2)))) | ...    4.0   4.0   5.0   
434     (p0 R p1) | FGp2                                      3.0   3.0   5.0   
437     ((p0 U p1) & FG!p2) | (GFp2 & (!p0 R !p1))            3.0   3.0  13.0   
440     F(p0 | ((p1 | p2) R (!p1 | p3)))                      2.0   2.0   3.0   
442     X(p0 | F(p0 & X(p1 R (p0 | p1))))                     5.0   5.0   3.0   
444     (p0 R p1) | F(p2 R p1) 

# Log parsing

In [29]:
from ltlcross_runner import parse_check_log

In [30]:
log_f = 'data_check/random_full.log'

In [31]:
bugs, bogus_f, tools = parse_check_log(log_f)

In [32]:
list(bogus_f.keys())[0]

480

In [33]:
bugs[480][:6]

['error: P0*N2 is nonempty',
 'error: P0*N4 is nonempty',
 'error: P0*N14 is nonempty',
 'error: P1*N2 is nonempty',
 'error: P1*N4 is nonempty',
 'error: P1*N14 is nonempty']

In [34]:
tools['P2']

"Rab4/bin/ltl2dra '((p0) R (p1)) R (F(p2))' | autfilt -S > 'lcr-o2-kSi8a2'"

In [35]:
import re
from ltlcross_runner import parse_log_tools
from ltlcross_runner import hunt_error_types

In [36]:
log_f = 'data_check/random_ltl3dra.log'

In [37]:
hunt_error_types(log_f)

({7: {'error: execution returned exit code 1.': ['N7', 'N9']},
  13: {'error: execution returned exit code 1.': ['N7', 'N9']},
  22: {'error: execution returned exit code 1.': ['N7', 'N9']},
  28: {'error: execution returned exit code 1.': ['N7', 'N9']},
  30: {'error: execution returned exit code 1.': ['N7', 'N9']},
  37: {'error: execution returned exit code 1.': ['N7', 'N9']},
  39: {'error: execution returned exit code 1.': ['N7', 'N9']},
  47: {'error: execution returned exit code 1.': ['N7', 'N9']},
  52: {'error: execution returned exit code 1.': ['N7', 'N9']},
  59: {'error: execution returned exit code 1.': ['N7', 'N9']},
  69: {'error: execution returned exit code 1.': ['N7', 'N9']},
  75: {'error: execution returned exit code 1.': ['N7', 'N9']},
  76: {'error: execution returned exit code 1.': ['N7', 'N9']},
  84: {'error: execution returned exit code 1.': ['N7', 'N9']},
  86: {'error: execution returned exit code 1.': ['N7', 'N9']},
  87: {'error: execution returned exit co

We should also check if Rabinizer match sizes in check runs and the _original_ runs.

### TODO:
 * Save formula indices externally